In [1]:
# General python libraires 
import os

# General DS libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

# Parallelized libraries 
import dask.dataframe as dd 

In [2]:
# Getting files 
path = 'C:/Users/redha.cherif_artefac/GitHub_perso/American_express_comp/input/'
train = dd.read_csv(os.path.join(path, 'train_data.csv'))
train_labels = dd.read_csv(os.path.join(path, 'train_labels.csv'))
test = dd.read_csv(os.path.join(path, 'test_data.csv'))

In [18]:
print(len(train))
print(len(test))

5531451
11363762


In [45]:
print(dd.compute(train.S_2.min(),train.S_2.max()))
print(dd.compute(test.S_2.min(), test.S_2.max()))
print('train: 12 months')
print('test: 18 months')

('2017-03-01', '2018-03-31')
('2018-04-01', '2019-10-31')
train: 12 months
test: 18 months


In [56]:
train.customer_ID

Dask Series Structure:
npartitions=256
    object
       ...
     ...  
       ...
       ...
Name: customer_ID, dtype: object
Dask Name: getitem, 512 tasks

In [52]:
train.tail()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
21602,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-11-05,0.979332,0.416013,0.020818,0.828199,0.003487,0.090743,0.005340,0.025139,...,NaN,NaN,NaN,0.006838,0.003680,0.000457,NaN,0.000905,0.001498,0.002774
21603,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-12-23,0.984907,0.296536,0.007209,0.812610,0.005904,0.079886,0.002243,0.023691,...,NaN,NaN,NaN,0.003310,0.007097,0.007857,NaN,0.002777,0.008225,0.008856
21604,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-01-06,0.983019,0.443984,0.013151,0.815422,0.003457,0.100503,0.002111,0.012343,...,NaN,NaN,NaN,0.009955,0.009994,0.001088,NaN,0.005693,0.006773,0.005566
21605,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-02-06,0.969861,0.442553,0.009855,1.003541,0.005117,0.101802,0.009930,0.008578,...,NaN,NaN,NaN,0.005541,0.006564,0.009883,NaN,0.008123,0.001168,0.003983
21606,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-03-14,0.982175,0.002474,0.000077,0.992880,0.000809,0.119165,0.003287,0.014092,...,NaN,NaN,NaN,0.007316,0.002888,0.006207,NaN,0.005110,0.003184,0.001914
